# Timbre transfer demo

Copyright 2020 InterDigital R&D and Télécom Paris.  
Original Author: Ondřej Cífka
Updates: Ali Dulaimi

## Install packages

## Local Setup Instructions

**For Google Colab**: Simply run all cells in order.

**For Local Environment**:
1. Install requirements
2. The pip uninstall/install steps are needed because:
   - The model requires specific PyTorch versions for compatibility
   - NumPy version conflicts can occur with DDSP dependencies
   - These steps ensure clean installation of compatible versions

**Important**: After running the NumPy downgrade cell, restart your kernel/runtime before proceeding.

In [1]:
!git clone https://github.com/cifkao/ss-vq-vae.git

Cloning into 'ss-vq-vae'...
remote: Enumerating objects: 432, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 432 (delta 58), reused 54 (delta 54), pack-reused 363 (from 1)
Receiving objects: 100% (432/432), 17.87 MiB | 15.49 MiB/s, done.
Resolving deltas: 100% (198/198), done.


In [2]:
!pip uninstall torch torchvision torchaudio accelerate -y -q

In [3]:
!pip install ./ss-vq-vae/src 'numba>0.57' ddsp -q

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.4/323.4 kB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 60.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Requested ddsp from https://files.pythonhosted.org/packages/46/34/521a494c2096fb43ecbafa08b7ccd092b53984cd8ace47f788eb2085f1ec/ddsp-3.5.0-py2.py3-none

⚠️ **Important**: After restarting the runtime, **DO NOT re-run the installation cells above**. Running them again may cause dependency conflicts and put you in an installation loop. Continue from the "Load the model" section.


In [4]:
!pip uninstall numpy
!pip install "numpy<2.0"

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/numpy-config
    /usr/local/lib/python3.11/dist-packages/numpy-2.0.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1-0352e75f.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99-934c22de.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libscipy_openblas64_-99b71e71.so
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? y
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 80.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.16.0 requires accelerate>=0.21.0, which is not installed.
opencv-pyt

## Inference with a pre-trained model

### Model Architecture Overview

This VQ-VAE (Vector Quantized Variational AutoEncoder) model performs timbre transfer through three main components:

### Key Components:
- **Content Encoder**: Extracts musical content (notes, rhythm) from the input audio while discarding timbre information
- **Style Encoder (RNN)**: Captures the timbral characteristics and playing style from the style audio using recurrent neural networks
- **Decoder Modules**: Reconstructs audio by combining the content representation with the style features, generating output with the target timbre

### Why Some Combinations Work Better:
- **Harmonic similarity**: Instruments with similar harmonic structures transfer more effectively
- **Spectral compatibility**: Instruments with overlapping frequency ranges produce cleaner results
- **Temporal characteristics**: Instruments with similar attack/decay patterns maintain better musical expression

## Download the model

#### Make sure to restart the session to load the correct NumPy version

In [4]:
logdir = 'ss-vq-vae/experiments/model'

In [6]:
!wget https://adasp.telecom-paris.fr/rc-ext/demos_companion-pages/vqvae_examples/ssvqvae_model_state.pt -O $logdir/model_state.pt

--2025-07-13 14:41:44--  https://adasp.telecom-paris.fr/rc-ext/demos_companion-pages/vqvae_examples/ssvqvae_model_state.pt
Resolving adasp.telecom-paris.fr (adasp.telecom-paris.fr)... 137.194.22.227, 2a04:8ec0:0:a::89c2:16e3
Connecting to adasp.telecom-paris.fr (adasp.telecom-paris.fr)|137.194.22.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 222788899 (212M)
Saving to: ‘ss-vq-vae/experiments/model/model_state.pt’

ss-vq-vae/experimen 100%[===================>] 212.47M  14.9MB/s    in 16s     

2025-07-13 14:42:02 (13.1 MB/s) - ‘ss-vq-vae/experiments/model/model_state.pt’ saved [222788899/222788899]



In [1]:
import numpy as np

np.__version__

'1.26.4'

## Load the model

In [2]:
import os

import confugue
from ddsp.colab import colab_utils
import librosa
import torch

from ss_vq_vae.models.vqvae_oneshot import Experiment

In [7]:
cfg = confugue.Configuration.from_yaml_file(os.path.join(logdir, 'config.yaml'))
exp = cfg.configure(Experiment, logdir=logdir, device='cpu')
exp.model.load_state_dict(torch.load(os.path.join(logdir, 'model_state.pt'), map_location=exp.device))
exp.model.train(False)

Model(
  (content_encoder): Sequential(
    (0): Conv1d(1025, 1024, kernel_size=(4,), stride=(2,), padding=(2,))
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1)
    (3): Conv1d(1024, 1024, kernel_size=(4,), stride=(2,), padding=(2,))
    (4): ResidualWrapper(
      (module): Sequential(
        (0): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): LeakyReLU(negative_slope=0.1)
        (2): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
      )
    )
    (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (vq): VQEmbedding(
    (embedding): Embedding(2048, 1024)
  )
  (style_encoder_1d): Sequential(
    (0): Conv1d(1025, 1024, kernel_size=(4,), stride=(2,))
    (1): ResidualWrapper(
      (module): Sequential(
        (0): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): L

In [8]:
INPUT_ROOT = 'https://adasp.telecom-paris.fr/rc-ext/demos_companion-pages/vqvae_examples/'
INPUT_URLS = {
    'Electric Guitar': INPUT_ROOT + 'real/content/UnicornRodeo_Maybe_UnicornRodeo_Maybe_Full_25_ElecGtr2CloseMic3.0148.mp3',
    'Electric Organ': INPUT_ROOT + 'real/style/AllenStone_Naturally_Allen%20Stone_Naturally_Keys-Organ-Active%20DI.0253.mp3',
    'Jazz Piano': INPUT_ROOT + 'real/style/MaurizioPagnuttiSextet_AllTheGinIsGone_MaurizioPagnuttiSextet_AllTheGinIsGone_Full_12_PianoMics1.08.mp3',
    'Synth': INPUT_ROOT + 'real/content/Skelpolu_TogetherAlone_Skelpolu_TogetherAlone_Full_13_Synth.0190.mp3',
    'Rhodes DI': INPUT_ROOT + 'real/content/Diesel13_ColourMeRed_Diesel13_ColourMeRed_Full_30_RhodesDI.0062.mp3',
    'Acoustic Guitar Lead': INPUT_ROOT + 'real/style/NikolaStajicFtVlasisKostas_Nalim_Nikola%20Stajic%20ft.%20Vlasis%20Kostas_Nalim_Acoustic%20Guitar-Lead-Ela%20M%20251.0170.mp3',
    'Bass Amp': INPUT_ROOT + 'real/content/HurrayForTheRiffRaff_LivingInTheCity_Hurray%20for%20the%20Riff%20Raff_Livin%20in%20the%20City_Bass-Amp-M82.0018.mp3',
    'Bass Bip': INPUT_ROOT + 'real/style/RememberDecember_CUNextTime_RememberDecember_CUNextTime_Full_11_Bass_bip.041.mp3',
    'SynthFX': INPUT_ROOT + 'real/content/MR0902_JamesElder_MR0902_JamesElder_Full_13_SynthFX1.163.mp3',
    'Electric Guitar Close': INPUT_ROOT + 'real/style/Fergessen_TheWind_Fergessen_TheWind_Full_17_SlecGtr3a_Close.146.mp3',
    'Rhodes NBATG': INPUT_ROOT + 'real/content/NickiBluhmAndTheGramblers_GoGoGo_NBATG%20-%20Rhodes%20-%20DI.098.mp3',
    'Keys DI Grace': INPUT_ROOT + 'real/style/JessicaChildress_SlowDown_SD%20KEYS-DI-GRACE.147.mp3',
    'Dulcimer': INPUT_ROOT + 'real/content/ButterflyEffect_PreachRightHere_ButterflyEffect_PreachRightHere_Full_16_Dulcimer2.076.mp3',
    'Strings Section': INPUT_ROOT + 'real/style/AngeloBoltini_ThisTown_AngeloBoltini_ThisTown_Full_47_Strings_SectionMic_Vln2.0181.mp3',
    'Mellotron': INPUT_ROOT + 'real/content/Triviul_Dorothy_Triviul_Dorothy_Full_07_Mellotron.120.mp3',
    'Acoustic Guitar CU': INPUT_ROOT + 'real/style/UncleDad_WhoIAm_legend-strings_AC%20GUITAR-3-CU29-SHADOWHILL.R.0106.mp3',
    'Fiddle': INPUT_ROOT + 'real/content/EndaReilly_CurAnLongAgSeol_EndaReilly_CurAnLongAgSeol_Full_10_Fiddle2.0163.mp3',
    'Violins': INPUT_ROOT + 'real/style/ScottElliott_AeternumVale_ScottElliott_AeternumVale_Full_41_Violins.0138.mp3',
    'Upright Bass': INPUT_ROOT + 'real/content/AbletonesBigBand_SongOfIndia_UPRIGHT%20BASS%20-%20ELA%20M%20260%20-%20Neve%2033102.136.mp3',
    'Taiko': INPUT_ROOT + 'real/style/CarlosGonzalez_APlaceForUs_CarlosGonzalez_APlaceForUs_Full_21_Taiko.0115.mp3',
    'Guitar 2': INPUT_ROOT + 'real/content/AllHandsLost_Ambitions_AllHandsLost_Ambitions_Full_Guitar%202.0292.mp3',
    'Alto Sax': INPUT_ROOT + 'real/style/SunshineGarciaBand_ForIAmTheMoon_zip5-outro-uke-shaker_OUTRO%20ALTO-251E-SSL6000E.0290.mp3',
    'Bass Close Mic': INPUT_ROOT + 'real/content/DonCamilloChoir_MarshMarigoldsSong_DonCamilloChoir_MarshMarigoldsSong_Full_08_BassCloseMic2.000.mp3',
    'Electric Guitar Distorted': INPUT_ROOT + 'real/style/EnterTheHaggis_TwoBareHands_25.%20Jubilee%20Riots%20-%202%20Bar%20Hands_ELE%20Guitars-Ignater-M81.160.mp3',
    'Bells': INPUT_ROOT + 'real/content/cryonicPAX_Melancholy_cryonicPAX_Melancholy_Full_10_Bells.0034.mp3',
    'Bass Mic 647': INPUT_ROOT + 'real/style/KungFu_JoyRide_40.%20Kung%20Fu%20-%20Joy%20ride_Bass-Mic-647.0090.mp3',
}

## Choose or record inputs

In [9]:
#@title Content input
# source of these audio files is on https://adasp.telecom-paris.fr/rc/demos_companion-pages/cifka-ss-vq-vae/
content_input = 'Violins'  #@param ["Record", "Electric Guitar", "Electric Organ", "Jazz Piano", "Synth", "Rhodes DI", "Acoustic Guitar Lead", "Bass Amp", "Bass Bip", "SynthFX", "Electric Guitar Close", "Rhodes NBATG", "Keys DI Grace", "Dulcimer", "Strings Section", "Mellotron", "Acoustic Guitar CU", "Fiddle", "Violins", "Upright Bass", "Taiko", "Guitar 2", "Alto Sax", "Bass Close Mic", "Electric Guitar Distorted", "Bells", "Bass Mic 647"]
record_seconds = 8 #@param {type:"number"}

if content_input == 'Record':
    a_content = colab_utils.record(seconds=record_seconds, sample_rate=exp.sr, normalize_db=0.1)
else:
    !wget {INPUT_URLS[content_input]} -O content_input.mp3
    a_content, _ = librosa.load('content_input.mp3', sr=exp.sr)
colab_utils.play(a_content, sample_rate=exp.sr)

--2025-07-13 14:42:13--  https://adasp.telecom-paris.fr/rc-ext/demos_companion-pages/vqvae_examples/real/style/ScottElliott_AeternumVale_ScottElliott_AeternumVale_Full_41_Violins.0138.mp3
Resolving adasp.telecom-paris.fr (adasp.telecom-paris.fr)... 137.194.22.227, 2a04:8ec0:0:a::89c2:16e3
Connecting to adasp.telecom-paris.fr (adasp.telecom-paris.fr)|137.194.22.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162004 (158K) [audio/mpeg]
Saving to: ‘content_input.mp3’

content_input.mp3   100%[===================>] 158.21K   312KB/s    in 0.5s    

2025-07-13 14:42:15 (312 KB/s) - ‘content_input.mp3’ saved [162004/162004]



In [10]:
#@title Style input
style_input = 'Electric Guitar Close'  #@param ["Record", "Electric Organ", "Jazz Piano", "Acoustic Guitar Lead", "Bass Bip", "Electric Guitar Close", "Keys DI Grace", "Strings Section", "Acoustic Guitar CU", "Violins", "Taiko", "Alto Sax", "Electric Guitar Distorted", "Bass Mic 647"]
record_seconds = 8 #@param {type:"number"}

if style_input == 'Record':
    a_style = colab_utils.record(seconds=record_seconds, sample_rate=exp.sr, normalize_db=0.1)
else:
    !wget {INPUT_URLS[style_input]} -O style_input.mp3
    a_style, _ = librosa.load('style_input.mp3', sr=exp.sr)
colab_utils.play(a_style, sample_rate=exp.sr)

--2025-07-13 14:42:25--  https://adasp.telecom-paris.fr/rc-ext/demos_companion-pages/vqvae_examples/real/style/Fergessen_TheWind_Fergessen_TheWind_Full_17_SlecGtr3a_Close.146.mp3
Resolving adasp.telecom-paris.fr (adasp.telecom-paris.fr)... 137.194.22.227, 2a04:8ec0:0:a::89c2:16e3
Connecting to adasp.telecom-paris.fr (adasp.telecom-paris.fr)|137.194.22.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162004 (158K) [audio/mpeg]
Saving to: ‘style_input.mp3’

style_input.mp3     100%[===================>] 158.21K   224KB/s    in 0.7s    

2025-07-13 14:42:27 (224 KB/s) - ‘style_input.mp3’ saved [162004/162004]



## Run the model

In [11]:
s_content = torch.as_tensor(exp.preprocess(a_content), device=exp.device)[None, :]
s_style = torch.as_tensor(exp.preprocess(a_style), device=exp.device)[None, :]
l_content, l_style = (torch.as_tensor([x.shape[2]], device=exp.device) for x in [s_content, s_style])
with torch.no_grad():
    s_output = exp.model(input_c=s_content, input_s=s_style,
                         length_c=l_content, length_s=l_style)
a_output = exp.postprocess(s_output.cpu().numpy()[0])
colab_utils.play(a_output, sample_rate=16000 * 2)

### Run the model on your own audio (first 8 seconds)


## 🎵 VQ-VAE Timbre Transfer Demo



In [ ]:
"""## Gradio Interface for Custom Audio Upload"""

import gradio as gr
import requests
import tempfile

# Separate content and style options based on URL paths
# This categorizes presets by looking for 'content' or 'style' keywords in the file paths
CONTENT_OPTIONS = [key for key in INPUT_URLS.keys() if any(word in INPUT_URLS[key] for word in ['content'])]
STYLE_OPTIONS = [key for key in INPUT_URLS.keys() if any(word in INPUT_URLS[key] for word in ['style'])]

# Add remaining items to both lists if they don't contain 'content' or 'style' in their paths
# This ensures all presets are available in both dropdowns for flexibility
for key in INPUT_URLS.keys():
    if key not in CONTENT_OPTIONS and key not in STYLE_OPTIONS:
        CONTENT_OPTIONS.append(key)
        STYLE_OPTIONS.append(key)

def load_audio_from_url(url, sr=None):
    """Load audio from URL by downloading to temporary file"""
    response = requests.get(url)
    # Create temporary file to store downloaded audio (deleted after use)
    with tempfile.NamedTemporaryFile(suffix='.mp3', delete=False) as tmp_file:
        tmp_file.write(response.content)
        tmp_file_path = tmp_file.name

    # Load audio using librosa and clean up temp file
    audio, _ = librosa.load(tmp_file_path, sr=sr)
    os.unlink(tmp_file_path)  # Delete temporary file
    return audio

def process_timbre_transfer(content_file, content_preset, style_file, style_preset, max_duration=8):
    """Process timbre transfer with uploaded files or presets

    Priority: uploaded files override preset selections
    """
    try:
        # Load content audio (musical notes/melody to preserve)
        # Priority: uploaded file > preset selection
        if content_file is not None:
            a_content, _ = librosa.load(content_file, sr=exp.sr)
        else:
            if content_preset and content_preset in INPUT_URLS:
                a_content = load_audio_from_url(INPUT_URLS[content_preset], sr=exp.sr)
            else:
                return None, "Please upload a content file or select a content preset"

        # Load style audio (timbre/texture to apply)
        # Same priority logic as content
        if style_file is not None:
            a_style, _ = librosa.load(style_file, sr=exp.sr)
        else:
            if style_preset and style_preset in INPUT_URLS:
                a_style = load_audio_from_url(INPUT_URLS[style_preset], sr=exp.sr)
            else:
                return None, "Please upload a style file or select a style preset"

        # Limit duration to prevent memory issues and long processing times
        max_samples = int(max_duration * exp.sr)
        if len(a_content) > max_samples:
            a_content = a_content[:max_samples]  # Truncate to max duration
        if len(a_style) > max_samples:
            a_style = a_style[:max_samples]

        # Preprocess: Convert audio to model input format (spectrograms)
        s_content = torch.as_tensor(exp.preprocess(a_content), device=exp.device)[None, :]
        s_style = torch.as_tensor(exp.preprocess(a_style), device=exp.device)[None, :]
        # Create length tensors for variable-length input handling
        l_content, l_style = (torch.as_tensor([x.shape[2]], device=exp.device) for x in [s_content, s_style])

        # Run model: Extract content features, extract style features, then recombine
        with torch.no_grad():  # Disable gradient computation for inference
            s_output = exp.model(input_c=s_content, input_s=s_style,
                               length_c=l_content, length_s=l_style)

        # Postprocess: Convert model output back to audio waveform
        a_output = exp.postprocess(s_output.cpu().numpy()[0])

        # Return audio in format expected by Gradio: (sample_rate, audio_array)
        return (exp.sr, a_output), "Transfer completed successfully!"

    except Exception as e:
        return None, f"Error: {str(e)}"

# Create Gradio interface with modern theme
with gr.Blocks(title="VQ-VAE Timbre Transfer", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🎵 VQ-VAE Timbre Transfer Demo

    Transfer the timbre (tone/texture) from one audio source to another while preserving the musical content.

    **Content**: Musical notes/melody that will be preserved
    **Style**: Instrument timbre/texture that will be applied
    """)

    # Two-column layout for content and style inputs
    with gr.Row():
        with gr.Column():
            gr.Markdown("### 🎼 Content Audio")
            content_file = gr.Audio(label="Upload Content Audio", type="filepath")
            content_preset = gr.Dropdown(
                choices=[""] + CONTENT_OPTIONS,  # Empty string allows no selection
                label="Or choose preset",
                value=""  # No default selection
            )

        with gr.Column():
            gr.Markdown("### 🎨 Style Audio")
            style_file = gr.Audio(label="Upload Style Audio", type="filepath")
            style_preset = gr.Dropdown(
                choices=[""] + STYLE_OPTIONS,
                label="Or choose preset",
                value="Electric Guitar Close"  # Default style selection
            )

    # Duration control to balance quality vs processing time
    max_duration = gr.Slider(1, 15, value=8, step=1, label="Max Duration (seconds)")

    process_btn = gr.Button("🚀 Transfer Timbre", variant="primary", size="lg")

    # Output section
    with gr.Row():
        output_audio = gr.Audio(label="🎵 Output Audio", interactive=False)
        status_msg = gr.Textbox(label="Status", interactive=False, max_lines=3)

    # Connect button click to processing function
    process_btn.click(
        fn=process_timbre_transfer,
        inputs=[content_file, content_preset, style_file, style_preset, max_duration],
        outputs=[output_audio, status_msg]
    )

# Launch with public sharing enabled and debug mode for development
demo.launch(share=True, debug=True, height=1000)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3629c487d475f93c9f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/gradio/processing_utils.py:753: UserWarning: Trying to convert audio automatically from float32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


## Troubleshooting

**Common Issues**:
- If you get NumPy version errors: Make sure to restart runtime after NumPy downgrade
- If installation fails: Don't re-run installation cells after restart
- If audio doesn't load: Check that URLs are accessible or try different presets
- If transfer sounds poor: Try different instrument combinations or adjust max duration

**Best Practices**:
- Use audio clips that are clear and not too noisy
- 8-second clips usually work best for quality vs. processing time
- Experiment with different style/content combinations